In [1]:
import ydata_profiling as yp

/Users/agpascoemacmini3/miniconda3/envs/project-template2/lib/python3.8/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/agpascoemacmini3/miniconda3/envs/project-template2/lib/python3.8/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow


In [2]:
table = pd.read_csv('../data/raw/Variables_IMSS.csv')

In [3]:
table['inference_date'] = table['inference_date'].astype('datetime64[ns]') 
table['target'] = table.target.astype("category")
table=table.drop(['Unnamed: 0'],axis=1)
table=table.drop(['curp'],axis=1)
table=table.drop(['Job_Maturity_Year'],axis=1)
table=table.drop(['Job_Maturity_Month'],axis=1)
table=table.drop(['Job_Maturity_Day'],axis=1)

from datetime import datetime
today = datetime.today()
target_date = pd.to_datetime(table['inference_date'])
days_past = (today - target_date).dt.total_seconds() / (60 * 60 * 24)
table['inference_date'] =  days_past

target_date = pd.to_datetime(table['last_job_end_date'])
days_past = (today - target_date).dt.total_seconds() / (60 * 60 * 24)
table['last_job_end_date'] =   days_past

table['ratio_Employer_yearMat'] = table['ratio_Employer_yearMat'].replace(np.inf, np.nan)
table['ratio_Employer_monMat'] = table['ratio_Employer_monMat'].replace(np.inf, np.nan)

table.drop_duplicates(inplace=True)

table.describe()

,inference_date,Job_Maturity_Week,Status_on_InferenceD,No_Job_history,Max_space_between_jobs,Mean_space_between_jobs,count_job_gap,last_job_end_date,last_job_base_salary,count_distinct_FederalEntity,Federal_entity_binary,Ratio_FedEnt_DistEmploy,ratio_Employer_yearMat,ratio_Employer_monMat,distinct_employer_registy,comp_entidades_curp,ratio_weeksmatured_18reportdate,ratio_weekscontributed_18reportdate,ratio_weekswithdrawn_18reportdate
count,83441.000000,82926.000000,83441.000000,83441.000000,80961.000000,80961.000000,82926.000000,82926.000000,82926.000000,82926.000000,83441.000000,82926.000000,82754.000000,81782.000000,82926.000000,82926.000000,82926.000000,82926.000000,82926.000000
mean,391.653924,336.839158,0.469266,0.006172,726.059930,175.672730,5.813955,1448.306414,318.303028,2.056146,0.570055,0.449908,2.975227,0.195527,7.223959,0.573475,0.330912,0.315774,0.026813
std,84.692883,321.267066,0.516639,0.078320,1117.372885,334.611237,15.222311,1927.803927,318.021110,1.289753,0.495071,0.295547,7.481346,0.267411,6.754753,0.494575,0.285686,0.278875,0.050841
min,267.616401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,203.616401,0.000000,1.000000,0.000000,0.030000,0.030000,0.000000,1.000000,0.000000,-1.186306,-0.771020,-0.252430
25%,313.616401,84.000000,0.000000,0.000000,3.000000,1.000000,1.000000,355.616401,149.590000,1.000000,0.000000,0.220000,0.730000,0.060000,3.000000,0.000000,0.098280,0.089669,0.000000
50%,386.616401,246.000000,0.000000,0.000000,297.000000,61.704545,3.000000,516.616401,209.040000,2.000000,1.000000,0.330000,1.390000,0.110000,5.000000,1.000000,0.303376,0.280000,0.000000
75%,455.616401,496.000000,1.000000,0.000000,952.000000,197.750000,7.000000,1705.616401,361.370000,3.000000,1.000000,0.570000,2.740000,0.220000,10.000000,1.000000,0.543701,0.519552,0.046005
max,589.616401,3292.000000,3.000000,1.000000,15330.000000,6524.000000,1242.000000,15839.616401,2405.500000,19.000000,1.000000,1.000000,300.000000,5.500000,118.000000,1.000000,2.238095,2.595238,0.751656


In [13]:
! pip install ipywidgets

ProfileReport = yp.ProfileReport(table, title="IMSS", explorative=True)
ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:

from pycaret.classification import *

In [5]:
table_exp = table.sample(frac=0.20, random_state=123)
table_unseen=table.drop(table_exp.index)
table_exp.shape,table_unseen.shape


((16688, 20), (66753, 20))

In [8]:
from pycaret.utils import set_experiment

# Set the experiment and specify the local file path
set_experiment('exp3', log_experiment=True, experiment_name_file='../mlruns')

ImportError: cannot import name 'set_experiment' from 'pycaret.utils' (/Users/agpascoemacmini3/miniconda3/envs/project-template2/lib/python3.8/site-packages/pycaret/utils/__init__.py)

In [6]:
dataset = setup( data = table_exp, target = 'target', session_id=123,
                 remove_outliers=True, outliers_threshold=0.05,
                 transformation=True, 
                 remove_multicollinearity=True,
                 log_experiment=True, experiment_name='exp4',
                 log_plots = True, 
                 fix_imbalance=True,
                 normalize=True, normalize_method='zscore')

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(16688, 20)"
4,Transformed data shape,"(22847, 19)"
5,Transformed train set shape,"(17840, 19)"
6,Transformed test set shape,"(5007, 19)"
7,Numeric features,19
8,Rows with missing values,4.1%
9,Preprocess,True


In [7]:
best= compare_models(sort='Recall')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6006,0.6135,0.5529,0.2589,0.3525,0.1155,0.1331,1.2640
ridge,Ridge Classifier,0.6007,0.0000,0.5520,0.2587,0.3521,0.1151,0.1327,0.4200
lda,Linear Discriminant Analysis,0.6001,0.6135,0.5502,0.2579,0.3511,0.1137,0.1310,0.4550
svm,SVM - Linear Kernel,0.5913,0.0000,0.5372,0.2495,0.3392,0.0982,0.1143,0.4720
knn,K Neighbors Classifier,0.6220,0.5537,0.3915,0.2300,0.2896,0.0554,0.0590,0.5060
nb,Naive Bayes,0.6785,0.6009,0.3628,0.2714,0.3036,0.1051,0.1092,0.4610
qda,Quadratic Discriminant Analysis,0.6746,0.5791,0.3310,0.2678,0.2835,0.0893,0.0888,0.4280
dt,Decision Tree Classifier,0.6653,0.5184,0.2758,0.2204,0.2449,0.0334,0.0337,0.4670
ada,Ada Boost Classifier,0.7582,0.6077,0.1657,0.2945,0.2114,0.0823,0.0870,0.7200
et,Extra Trees Classifier,0.7788,0.5899,0.0944,0.3000,0.1431,0.0549,0.0670,2.2280


In [ ]:
lr_model = create_model("lr")

In [ ]:
plot_model(lr_model, plot='pr')

In [ ]:
plot_model(lr_model, plot='confusion_matrix')